# Ge - Visão Computacional
# Alunos: 
Leila Biggi e Lucas Heron

## Importações e úteis

In [1]:
import cv2
import pandas as pd
import os

In [4]:
healthy = '../Test/Healthy/'
powdery = '../Test/Powdery/'
rust = '../Test/Rust/'


healthy_images = [(os.path.join(healthy, f), 0) for f in os.listdir(healthy) if f.endswith('.jpg')]
powdery_images = [(os.path.join(powdery, f), 1) for f in os.listdir(powdery) if f.endswith('.jpg')]
rust_images = [(os.path.join(rust, f), 2) for f in os.listdir(rust) if f.endswith('.jpg')]

all_images = healthy_images + powdery_images + rust_images

df = pd.DataFrame(all_images, columns=['image_path', 'label'])

df = df.sample(frac=1).reset_index(drop=True)

In [7]:
df.to_csv('../Test/plant_disease.csv', index=False)